This notebook is a test notebook from the NIGMS ME-INBRE project that will run an RNAseq workflow.

In [22]:
# Import necessary packages
import subprocess as sp
import os

In [23]:
# Test out the following commands. First, set up your directory structure and download files with this command. 
# You can peek at what it is doing by typing cat scripts/setup.sh.
# Note that since this is Python script we have to wrap our bash commands in supprocess calls
shell_call = "sh scripts/setup.sh"
proc_shell = sp.call(shell_call,shell=True)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  280M  100  280M    0     0  35.1M      0  0:00:07  0:00:07 --:--:-- 32.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  280M  100  280M    0     0  32.6M      0  0:00:08  0:00:08 --:--:-- 32.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4982k  100 4982k    0     0  17.5M      0 --:--:-- --:--:-- --:--:-- 17.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    12  100    12    0     0    144      0 --:--:-- --:--:-- --:--:--   144
  % Total    % Received % Xferd  Average Speed   Tim

In [24]:
# Everything should now be downloaded
# Let's trim our raw reads
# First make a list of all files we want to trim (here we only have two)
files = [i.strip() for i in open("scripts/samples")]
# Then create a loop for each file
# use 'f' string to insert the {f} variable for each sample
for f in files:
    trim_call = f"trimmomatic SE -threads 4 data/raw_fastq/{f}.fastq data/trimmed/{f}_trimmed.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36"
    proc_shell = sp.call(trim_call,shell=True)
    


TrimmomaticSE: Started with arguments:
 -threads 4 data/raw_fastq/SRR13349122.fastq data/trimmed/SRR13349122_trimmed.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 2000000 Surviving: 2000000 (100.00%) Dropped: 0 (0.00%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 4 data/raw_fastq/SRR13349123.fastq data/trimmed/SRR13349123_trimmed.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detec

In [25]:
# Let's align reads to the reference genome
# Asign Ref variable
REF='data/reference/M_chelonae_NZ_CP007220.fasta'
for f in files:
    bwa_call = f"bwa mem -t 4 -R '@RG\\tID:{f}\\tSM:{f}' {REF} data/trimmed/{f}_trimmed.fastq > data/aligned/{f}.sam"
    proc_bwa = sp.call(bwa_call,shell=True)

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 784336 sequences (40000070 bp)...
[M::process] read 784338 sequences (40000056 bp)...
[M::mem_process_seqs] Processed 784336 reads in 13.867 CPU sec, 3.338 real sec
[M::process] read 431326 sequences (21996952 bp)...
[M::mem_process_seqs] Processed 784338 reads in 14.448 CPU sec, 3.320 real sec
[M::mem_process_seqs] Processed 431326 reads in 9.044 CPU sec, 1.993 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -t 4 -R @RG\tID:SRR13349122\tSM:SRR13349122 data/reference/M_chelonae_NZ_CP007220.fasta data/trimmed/SRR13349122_trimmed.fastq
[main] Real time: 9.629 sec; CPU: 38.320 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 784318 sequences (40000061 bp)...
[M::process] read 784318 sequences (40000057 bp)...
[M::mem_process_seqs] Processed 784318 reads in 14.831 CPU sec, 3.581 real sec
[M::process] read 431364 sequences (21999487 bp)...
[M::mem_process_seqs] Processed 784318 reads in 16.200 CPU

In [29]:
# Convert our sam file to binary form (bam)
for f in files:
    samtools_call = f"samtools view -S -b data/aligned/{f}.sam > data/aligned/{f}.bam"
    proc_samtools = sp.call(samtools_call,shell=True)

In [30]:
# Lets see how well our fastq files mapped to the reference genome
for f in files:
    depth_call = f"samtools flagstat data/aligned/{f}.bam > data/stats/{f}.flagstat.out"
    proc_depth = sp.call(depth_call,shell=True)

# Lets view the output
for f in files:
    print(f'mapping stats for {f}')
    catfile_call = f"cat data/stats/{f}.flagstat.out"
    proc_catfile = sp.call(catfile_call,shell=True)
    print ('\n'*2)


mapping stats for SRR13349122
2000005 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
5 + 0 supplementary
0 + 0 duplicates
1985433 + 0 mapped (99.27% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)



mapping stats for SRR13349123
2000003 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
3 + 0 supplementary
0 + 0 duplicates
1990036 + 0 mapped (99.50% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)





In [31]:
# Sort our BAM files so they are in the same order as the reference genome
for f in files:
    sort_call = f"gatk SortSam -I data/aligned/{f}.bam -O data/sorted/{f}.bam -SO coordinate"
    proc_sort = sp.call(sort_call,shell=True)

Using GATK jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar SortSam -I data/aligned/SRR13349122.bam -O data/sorted/SRR13349122.bam -SO coordinate
14:51:53.357 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.dylib
[Wed Aug 18 14:51:53 EDT 2021] SortSam --INPUT data/aligned/SRR13349122.bam --OUTPUT data/sorted/SRR13349122.bam --SORT_ORDER coordinate --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --GA4GH_CLIENT_SECRETS client_secrets.json --h

Tool returned:
0


Using GATK jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar SortSam -I data/aligned/SRR13349123.bam -O data/sorted/SRR13349123.bam -SO coordinate
14:52:09.487 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.dylib
[Wed Aug 18 14:52:09 EDT 2021] SortSam --INPUT data/aligned/SRR13349123.bam --OUTPUT data/sorted/SRR13349123.bam --SORT_ORDER coordinate --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 2 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --GA4GH_CLIENT_SECRETS client_secrets.json --h

Tool returned:
0


[Wed Aug 18 14:52:23 EDT 2021] picard.sam.SortSam done. Elapsed time: 0.23 minutes.
Runtime.totalMemory()=1936719872


In [33]:
# Mark Duplicates both from sequencing (optical) and the wetlab (PCR duplicates)
for f in files:
    dups_call = f"gatk MarkDuplicates I=data/sorted/{f}.bam O=data/mkdups/{f}.bam M=data/mkdups/{f}.metrics.txt"
    proc_dups = sp.call(dups_call,shell=True)

Using GATK jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar MarkDuplicates I=data/sorted/SRR13349122.bam O=data/mkdups/SRR13349122.bam M=data/mkdups/SRR13349122.metrics.txt
INFO	2021-08-18 14:52:53	MarkDuplicates	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    MarkDuplicates -I data/sorted/SRR13349122.bam -O data/mkdups/SRR13349122.bam -M data/mkdups/SRR13349122.metrics.txt
**********


14:52:53.702 INFO  NativeLibra

Tool returned:
0


Using GATK jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar MarkDuplicates I=data/sorted/SRR13349123.bam O=data/mkdups/SRR13349123.bam M=data/mkdups/SRR13349123.metrics.txt
INFO	2021-08-18 14:53:07	MarkDuplicates	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    MarkDuplicates -I data/sorted/SRR13349123.bam -O data/mkdups/SRR13349123.bam -M data/mkdups/SRR13349123.metrics.txt
**********


14:53:07.913 INFO  NativeLibra

Tool returned:
0


In [34]:
# Index BAMS
for f in files:
    index_call = f"samtools index -@ 4 data/mkdups/{f}.bam"
    proc_index = sp.call(index_call,shell=True)

In [35]:
# Call variants using GATK HaplotypeCAller
for f in files:
    var_call = f"gatk HaplotypeCaller -I data/mkdups/{f}.bam -O data/calls/{f}.vcf -R {REF} --output-mode EMIT_VARIANTS_ONLY"
    proc_var = sp.call(var_call,shell=True)

Using GATK jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar HaplotypeCaller -I data/mkdups/SRR13349122.bam -O data/calls/SRR13349122.vcf -R data/reference/M_chelonae_NZ_CP007220.fasta --output-mode EMIT_VARIANTS_ONLY
14:53:31.414 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/Users/kyoconnell/miniconda3/share/gatk4-4.2.0.0-1/gatk-package-4.2.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.dylib
14:53:31.549 INFO  HaplotypeCaller - ------------------------------------------------------------
14:53:31.549 INFO  HaplotypeCaller - The Genome Analysis Toolkit (GATK) v4.2.0.0
14:53:31.549 INFO  HaplotypeCaller - For support and documentation go to https://software.

14:53:50.882 WARN  InbreedingCoeff - InbreedingCoeff will not be calculated at position NZ_CP007220.1:240681 and possibly subsequent; at least 10 samples must have called genotypes
14:53:58.922 INFO  ProgressMeter - NZ_CP007220.1:2483573              0.2                 10920          65487.3
14:54:04.887 INFO  HaplotypeCaller - 671 read(s) filtered by: MappingQualityReadFilter 
0 read(s) filtered by: MappingQualityAvailableReadFilter 
0 read(s) filtered by: MappedReadFilter 
0 read(s) filtered by: NotSecondaryAlignmentReadFilter 
1881338 read(s) filtered by: NotDuplicateReadFilter 
0 read(s) filtered by: PassesVendorQualityCheckReadFilter 
0 read(s) filtered by: NonZeroReferenceLengthAlignmentReadFilter 
0 read(s) filtered by: GoodCigarReadFilter 
0 read(s) filtered by: WellformedReadFilter 
1882009 total reads filtered
14:54:04.887 INFO  ProgressMeter - NZ_CP007220.1:5029685              0.3                 22151          83222.3
14:54:04.887 INFO  ProgressMeter - Traversal complete.

In [36]:
# Let's see how many variants we have! 
# How many variants do we have? 
for f in files:
    vcf_call = f"vcftools --vcf data/calls/{f}.vcf"
    proc_vcf = sp.call(vcf_call,shell=True)


VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf data/calls/SRR13349122.vcf

After filtering, kept 1 out of 1 Individuals
After filtering, kept 22 out of a possible 22 Sites
Run Time = 0.00 seconds

VCFtools - 0.1.16
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf data/calls/SRR13349123.vcf

After filtering, kept 1 out of 1 Individuals
After filtering, kept 18 out of a possible 18 Sites
Run Time = 0.00 seconds
